In [28]:
import numpy as np
import scipy.io
import numpy as np
from scipy.io import savemat, loadmat

from transformers import AutoTokenizer, AutoModel 
import torch 

In [29]:
#read strings from txt file while removing \n
with open('./stimuli/stimuli_180concepts.txt', 'r') as f:
    concepts180 = f.readlines()
    concepts180 = [line.strip() for line in concepts180]

with open('./stimuli/stimuli_243sentences_dereferencedpronouns.txt', 'r') as f:
    concepts243 = f.readlines()
    concepts243 = [line.strip() for line in concepts243]

with open('./stimuli/stimuli_384sentences_dereferencedpronouns.txt', 'r') as f:
    concepts384 = f.readlines()
    concepts384 = [line.strip() for line in concepts384]

In [30]:

# Load the MATLAB file
main180 = scipy.io.loadmat('./glove_data/180_concepts.mat')
real180 = scipy.io.loadmat('./glove_data/180_concepts_real.mat')
real243 = scipy.io.loadmat('./glove_data/243_sentences_real.mat')
real384 = scipy.io.loadmat('./glove_data/384_sentences_real.mat')

print(real180.keys())
print(real243.keys())
print(real384.keys())
print(np.array(concepts180).shape)

dict_keys(['__header__', '__version__', '__globals__', 'data'])
dict_keys(['__header__', '__version__', '__globals__', 'data'])
dict_keys(['__header__', '__version__', '__globals__', 'data'])
(180,)


In [32]:
tokeniser = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

def generate_BERT_pooled(stimuli):
    # Tokenise the sentences
    BERT_pooled = []
    for s in stimuli:
        tokenised_sentence = tokeniser.encode(s, add_special_tokens=True)
        input_ids = torch.tensor(tokenised_sentence).unsqueeze(0)  # Batch size 1
        with torch.no_grad():
            outputs = model(input_ids)
            pooled_embedding = torch.mean(outputs.last_hidden_state,dim=1)
            BERT_pooled.append(pooled_embedding)

    return np.concatenate(BERT_pooled, axis=0)

BERT180 = generate_BERT_pooled(concepts180)
BERT243 = generate_BERT_pooled(concepts243)
BERT384 = generate_BERT_pooled(concepts384)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
savemat('./BERT_data/180_concepts.mat', {'words':np.array(concepts180), 'data': BERT180})
savemat('./BERT_data/180_concepts_real.mat', {'data': BERT180})
savemat('./BERT_data/243_sentences_real.mat', {'data': BERT243})
savemat('./BERT_data/384_sentences_real.mat', {'data': BERT384})

In [40]:
my_loaded_array = loadmat('./BERT_data/384_sentences_real.mat')
my_loaded_array['data'].shape

(384, 768)